In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
import io

GA_2015=pd.read_excel('data/states/Georgia/Georgia2015.xlsx')
GA_2016=pd.read_excel('data/states/Georgia/Georgia2016.xlsx')
GA_2017=pd.read_excel('data/states/Georgia/Georgia2017.xlsx')
GA_2018=pd.read_excel('data/states/Georgia/Georgia2018.xlsx')
GA_2019=pd.read_excel('data/states/Georgia/Georgia2019.xlsx')
GA_2020=pd.read_excel('data/states/Georgia/Georgia2020.xlsx')
georgia_df = GA_2015
georgia_df = pd.concat([georgia_df, GA_2016]).reset_index(drop=True) 
georgia_df = pd.concat([georgia_df, GA_2017]).reset_index(drop=True) 
georgia_df = pd.concat([georgia_df, GA_2018]).reset_index(drop=True)  
georgia_df = pd.concat([georgia_df, GA_2019]).reset_index(drop=True) 
georgia_df = pd.concat([georgia_df, GA_2020]).reset_index(drop=True) 

county_pop_df = pd.read_csv("data/states/county_pop.csv", encoding = "utf-8",error_bad_lines=True)
county_pop_df = county_pop_df[county_pop_df['STNAME'] == 'Georgia']

county_fips_df = pd.read_csv("data/states/county_fips.csv", error_bad_lines=True)
county_fips_df = county_fips_df[county_fips_df['STNAME'] == 'Georgia']
county_fips_df["CTYNAME"] = county_fips_df["CTYNAME"].str.upper()
county_fips_dict = dict(zip(county_fips_df.CTYNAME,county_pop_df.COUNTY))


In [2]:

georgia_constant_df = georgia_df.copy()
georgia_constant_df['state_FIPS'] = 13
georgia_constant_df['County'] = 0
georgia_constant_df['population_adjustment'] = 'Constant'
georgia_constant_df['pop'] = 0
georgia_constant_df["total_deaths_per100k"] = 0
georgia_constant_df["excess_deaths_per100k"] = 0.0
county_2017_pop= dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
pop_100k = 100000.0
for index, row in georgia_constant_df.iterrows():
    county_name = row['Place of Death County']
    pop = county_2017_pop[county_fips_dict[county_name]]
    georgia_constant_df.at[index, 'County'] = county_fips_dict[county_name]
    georgia_constant_df.at[index,'pop'] = pop
    georgia_constant_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = georgia_constant_df[georgia_constant_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in georgia_constant_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        georgia_constant_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - (avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]['avg_deaths']),4)       


In [3]:
county_year_dict = dict()
county_year_dict[2014] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2014))
county_year_dict[2015] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2015))
county_year_dict[2016] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2016))
county_year_dict[2017] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
county_year_dict[2018] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2018))
county_year_dict[2019] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2019))

georgia_lagged_df = georgia_df.copy()
georgia_lagged_df['state_FIPS'] = 13
georgia_lagged_df['County'] = 0
georgia_lagged_df['population_adjustment'] = 'Lagged'
georgia_lagged_df['pop'] = 0
georgia_lagged_df["total_deaths_per100k"] = 0
georgia_lagged_df["excess_deaths_per100k"] = 0.0
pop_100k = 100000.0
for index, row in georgia_lagged_df.iterrows():
    year = row['Year'] - 1  
    county_name = row['Place of Death County']
    county_pop_dict = county_year_dict[year]
    pop = county_pop_dict[county_fips_dict[county_name]]
    georgia_lagged_df.at[index, 'County'] = county_fips_dict[county_name]
    georgia_lagged_df.at[index,'pop'] = pop
    georgia_lagged_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = georgia_lagged_df[georgia_lagged_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in georgia_lagged_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        georgia_lagged_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - (avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]['avg_deaths']),4)       

georgia_merged_df = pd.concat([georgia_constant_df, georgia_lagged_df])
georgia_merged_df = georgia_merged_df.sort_values(['County', 'Week', 'population_adjustment','Year'])


In [4]:
georgia_panel_df = pd.DataFrame(columns=['State_FIPS','County_FIPS','Year', 'Week','Population_Adjustment', 
                                        'Deaths_Per100k', 'Excess_Deaths_Per100k'])        
georgia_panel_df['State_FIPS']  = georgia_merged_df['state_FIPS']
georgia_panel_df['County_FIPS'] = georgia_merged_df['County']
georgia_panel_df['Year']  = georgia_merged_df['Year']
georgia_panel_df['Week'] = georgia_merged_df['Week']
georgia_panel_df['Population_Adjustment'] = georgia_merged_df['population_adjustment']
georgia_panel_df['Deaths_Per100k'] = georgia_merged_df['total_deaths_per100k']
georgia_panel_df['Excess_Deaths_Per100k'] = round(georgia_merged_df['excess_deaths_per100k'],4)

In [5]:
georgia_panel_df.to_csv("georgia_panel.csv",index=False)